# Analyzing the logs

In [ ]:
# Run from the top of the repoository
%cd ..

In [ ]:
# TODO: Put actual results here

boom_throughput_factor = "TBD"
kafka_ingest_strate_mbps = "XXXXX"
kafka_ingest_factor = "XXXXX"

template = r"""% This file was automatically generated; edits will be overwritten!
\newcommand{\boomthroughputfactor}{BOOM_THROUGHPUT_FACTOR}
\newcommand{\kakfaingestratembps}{KAFKA_INGEST_STRATE_MBPS}
\newcommand{\kakfaingestfactor}{KAFKA_INGEST_FACTOR}
"""

# This might be better with an f-string, but this is TeX and who wants to deal
# with escaping all the braces?
template = (
    template.replace("BOOM_THROUGHPUT_FACTOR", boom_throughput_factor)
    .replace("KAFKA_INGEST_STRATE_MBPS", kafka_ingest_strate_mbps)
    .replace("KAFKA_INGEST_FACTOR", kafka_ingest_factor)
)

with open("paper/results.tex", "w") as f:
    f.write(template)